### (1)获得ADOS的dataset class
把获得的7frames转成tensor，并储存为list格式的sampleLib。

储存的tensor已经对图片进行了transform


In [1]:
import torch
import cv2
import os
from torchvision import transforms

In [ ]:
def video_tensor(input_video):
    transform=transforms.Compose([  transforms.ToPILImage(),
    transforms.Resize(256),  
                                transforms.CenterCrop(224),  # crop到224
                                transforms.ToTensor()])
    tensor_list = []
    cap = cv2.VideoCapture(input_video)
    for i in range(int(cap.get(7))):
        ret, frame = cap.read()
        frame_trans = transform(frame)
        frame_trans = torch.unsqueeze(frame_trans, 0)
        tensor_list.append(frame_trans)
    video_tensor = torch.cat(tensor_list,dim=0) 
    return video_tensor

In [ ]:
# /AsdData/pany/ados_mix_folder/11/20190516-黄翰_466.avi NonSmile

if __name__=="__main__":
    
    root_path = '/AsdData/pany/ados_mix_folder/' 
    label_dict = {'Smile':0,'NonSmile':1}
    for i in range(12):
        file = os.path.join(root_path,'{}/label.txt'.format(i))      
        video_list = []
        
        for line in open(file):  
            [video_path,label_class] = line.split()
            if label_class in ['Smile','NonSmile'] :
                tensor_list  = video_tensor(video_path)
                label = label_dict[label_class]
                if int(tensor_list.shape[0]) !=7:
                    print(video_path)
                video_list.append([tensor_list,label])
        torch.save(video_list, '/AsdData/pany/RSS_checkpoints/pt14/ados_video_tensor_list_{}.pkl'.format(i))

In [ ]:
if __name__=="__main__":
    
    root_path = '/AsdData/pany/ados_mix_folder/' 
    label_dict = {'Smile':0,'NonSmile':1}
    for i in range(12):
        file = os.path.join(root_path,'{}/label.txt'.format(i))      
        video_list = []
        
        for line in open(file):  
            [video_path,label_class] = line.split()
            if label_class in ['Smile','NonSmile'] :
                tensor_list  = video_tensor(video_path)
                label = label_dict[label_class]
                if int(tensor_list.shape[0]) !=7:
                    print(video_path)
                video_list.append([tensor_list,label])
        torch.save(video_list, '/AsdData/pany/RSS_checkpoints/pt14/ados_video_tensor_list_{}.pkl'.format(i))

In [ ]:
list_1 = torch.load('/AsdData/pany/RSS_checkpoints/pt14/ados_video_tensor_list_1.pkl')
list_2 = torch.load('/AsdData/pany/RSS_checkpoints/pt14/ados_video_tensor_list_1.pkl')

In [ ]:
list_1.extend(list_2)

In [ ]:
len(list_2)

In [ ]:
if __name__=="__main__":
    
    video_list = []
    for i in range(11):
        file_path = '/AsdData/pany/RSS_checkpoints/pt14/ados_video_tensor_list_{}.pkl'.format(i)    
        list_i = torch.load(file_path)
        video_list.extend(list_i)
    print(len(video_list))
    torch.save(video_list, '/AsdData/pany/RSS_checkpoints/pt14/ados_video_tensor_list_no_11.pkl')

### (2)导入pretrained模型

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
from __future__ import division
# import dlib
import time
import sys
import torch.utils.data as data
import cv2
import os
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.autograd import Variable  
from PIL import Image
from torch.nn import functional as F
from torch.utils.data import Dataset 
import numpy as np
from torch.utils import data as Data
from tqdm import tqdm
import time

import torchvision.models as models

from ops.basic_ops import ConsensusModule
from tfennet14 import *
# get_ipython().run_line_magic('matplotlib', 'notebook')

import visdom
import warnings

warnings.filterwarnings('ignore')

In [3]:
vis =  visdom.Visdom(env=u'pytorch14',use_incoming_socket=False)

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


In [4]:
class Mydataset(data.Dataset):
    '''
    the format of sampleLib:[tensor,label],[tensor,label],[tensor,label].......
    
    '''
    def __init__(self,sampleLib):
        self.sampleLib=sampleLib
#         self.transform = transforms.Compose([
#                                                transforms.ToTensor()
#                                            ])
    def __getitem__(self,index):
        [sample,label] = self.sampleLib[index]
        return sample, label

    def __len__(self):
        return len(self.sampleLib)

In [19]:
def transfer_model(pretrained_file, model):
    '''
    只导入pretrained_file部分模型参数
    tensor([-0.7119,  0.0688, -1.7247, -1.7182, -1.2161, -0.7323, -2.1065, -0.5433,-1.5893, -0.5562]
    update:
        D.update([E, ]**F) -> None.  Update D from dict/iterable E and F.
        If E is present and has a .keys() method, then does:  for k in E: D[k] = E[k]
        If E is present and lacks a .keys() method, then does:  for k, v in E: D[k] = v
        In either case, this is followed by: for k in F:  D[k] = F[k]
    :param pretrained_file:
    :param model:
    :return:
    '''
    pretrained_dict = torch.load(pretrained_file)  # get pretrained dict
    model_dict = model.state_dict()  # get model dict
    # 在合并前(update),需要去除pretrained_dict一些不需要的参数
    pretrained_dict = transfer_state_dict(pretrained_dict, model_dict)
    model_dict.update(pretrained_dict)  # 更新(合并)模型的参数
    model.load_state_dict(model_dict)

def transfer_state_dict(pretrained_dict, model_dict):
    '''
    根据model_dict,去除pretrained_dict一些不需要的参数,以便迁移到新的网络
    url: https://blog.csdn.net/qq_34914551/article/details/87871134
    :param pretrained_dict:
    :param model_dict:
    :return:
    '''
    # state_dict2 = {k: v for k, v in save_model.items() if k in model_dict.keys()}
    state_dict = {}
    for k, v in pretrained_dict.items():
        if k in model_dict.keys():
            # state_dict.setdefault(k, v)
            state_dict[k] = v
        else:
            print("Missing key(s) in state_dict :{}".format(k))
    return state_dict


In [24]:
def adjust_learning_rate(optimizer, epoch, lr):
    """Sets the learning rate to the initial LR decayed by 10 every SSTEP epochs"""
    lr = lr * (0.01 ** (epoch // sstep))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def validation(test_loader,net):
    '''
    注意，此处有几个修改，test_y_list的写法修改过了
    '''
    pred_list = []
    test_y_list = []
    for i in range(len(test_dataset)):
        test_y_list.append(test_dataset[i][1])
    #     test_loader.size
    for step, (test_x,test_y) in enumerate(test_loader):
#         test_x = test_x.to(DEVICE)
        test_x = test_x.cuda()
        test_y =test_y.numpy()

        test_output = net(test_x)                   
        pred_y = torch.max(test_output, 1)[1].data.cpu().numpy()

        pred_list +=  list(pred_y  )

    pred_ay = np.array(pred_list)
    test_y_ay = np.array(test_y_list)

    accu = float((pred_ay ==test_y_ay).astype(int).sum())/test_y_ay.size
    return accu

In [22]:
# transfer_model(net_2,net_6_pretrain)

In [6]:
net_2 = TFEN_2()
net_6_pretrain = torch.load('torch14_oulu_netall.pkl')
net_6_dict= net_6_pretrain.state_dict()

net_2_dict = net_2.state_dict()
net_2_dict_trans = transfer_state_dict(net_6_para_state, net_2_dict)

net_2_dict.update(net_2_dict_trans)
net_2.load_state_dict(net_2_dict)

In [ ]:
# 10-fold cross-validation
time_start=time.time()


loss_10 = []
accu_10 = []
net_10 = []
group_list = [11]
for i in group_list:
    BATCH_SIZE =16
    print('start loading data')
    train_dataset = torch.load('/AsdData/pany/RSS_checkpoints/pt14/ados_video_tensor_list_no_11.pkl')
    
    train_loader = Data.DataLoader(
        dataset=train_dataset,      
        batch_size=BATCH_SIZE,      
        shuffle = False,               
    )
    test_dataset = torch.load('/AsdData/pany/RSS_checkpoints/pt14/ados_video_tensor_list_{}.pkl'.format(i))
    test_loader = Data.DataLoader(
        dataset=test_dataset,     
        batch_size=BATCH_SIZE,     
        shuffle = False,               
    )
    print('finish loading data')
    net = net_2
#     net = torch.nn.DataParallel(net.cuda(), device_ids=[3])
    net = net.cuda()
    
    EPOCH = 400
    LR = 0.001
    MOMENTUM = 0.9
    WEIGHT_DECAY = 0.0001
    sstep = 70

    optimizer = torch.optim.SGD(net.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    loss_func = torch.nn.CrossEntropyLoss()
    loss_his = []
    accu_list = []
    
    
    for epoch in range(EPOCH):
        
#         print('====FOLDER:{}====EPOCH:{}/{}===='.format(i,epoch,EPOCH))
        
        adjust_learning_rate(optimizer, epoch, LR)
        
        for step, (batch_x,batch_y) in enumerate(train_loader):

            batch_x = batch_x.cuda()
            batch_y = batch_y.cuda()

            out = net(batch_x)
            loss = loss_func(out,batch_y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_his.append(loss.data.cpu().numpy())
                        
            if step % 50 == 0:
                accuracy = validation(test_loader,net)
                accu_list.append(accuracy)
                print('Step:{}, Accuracy Rate:{}'.format(step,accuracy))
                
                
                
                if accuracy == max(accu_list):
                    net_best = net
                
                
                
                x= torch.Tensor([len(accu_list)])
                y=torch.Tensor([accuracy])
                vis.line(X=x, Y=y, 
                         win='ados_finetune_{}'.format(i), 
                    opts=dict(
                        title='ados_finetune_{}'.format(i),
                        width=800,
                        height=800,
                             ),
                         update='append' if len(accu_list)>0 else None)



    loss_10.append(loss_his)
    accu_10.append(accu_list)
    net_10.append(net_best)
    
    


# In[ ]:



time_end=time.time()
m, s = divmod(time_end-time_start, 60)
h, m = divmod(m, 60)
print ("%02d:%02d:%02d" % (h, m, s))


# In[ ]:


m, s = divmod(time_end-time_start, 60)
h, m = divmod(m, 60)
print ("%02d:%02d:%02d" % (h, m, s))


# In[ ]:


torch.save(net_10,'torch14_adjustlr0.01_sstep70_finetune_ados.pkl')
torch.save(accu_10,'torch14_adjustlr0.01_sstep70_finetune_ados.pkl')
torch.save(loss_10,'torch14_adjustlr0.01_sstep70_finetune_ados.pkl')
print(time.strftime('torch14_adjustlr0.01_sstep70_finetune_ados.pkl ===> %Y-%m-%d %H:%M:%S', time.localtime()))



start loading data
finish loading data
Step:0, Accuracy Rate:0.4607142857142857
Step:50, Accuracy Rate:0.65
Step:100, Accuracy Rate:0.6226190476190476
Step:150, Accuracy Rate:0.7571428571428571
Step:200, Accuracy Rate:0.775
Step:250, Accuracy Rate:0.7345238095238096
Step:300, Accuracy Rate:0.8178571428571428
Step:350, Accuracy Rate:0.8214285714285714
Step:400, Accuracy Rate:0.8095238095238095
Step:450, Accuracy Rate:0.7511904761904762
Step:500, Accuracy Rate:0.8583333333333333
Step:550, Accuracy Rate:0.8571428571428571
Step:600, Accuracy Rate:0.8726190476190476
Step:0, Accuracy Rate:0.6666666666666666
Step:50, Accuracy Rate:0.85
Step:100, Accuracy Rate:0.8321428571428572
Step:150, Accuracy Rate:0.8642857142857143
Step:200, Accuracy Rate:0.844047619047619
Step:250, Accuracy Rate:0.8666666666666667
Step:300, Accuracy Rate:0.8464285714285714
Step:350, Accuracy Rate:0.8714285714285714
Step:400, Accuracy Rate:0.8464285714285714
Step:450, Accuracy Rate:0.8726190476190476
Step:500, Accuracy R